In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())
x_vals = data['x']
y_vals = data['y']

real_cov = statistics.covariance(x_vals, y_vals)
x_mean = statistics.mean(x_vals)
y_mean = statistics.mean(y_vals)
print("cov: ",real_cov )
print("x mean: ", x_mean)
print("y mean: ", y_mean)


create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())
dummy_x_vals = dummy_data['x']
dummy_y_vals = dummy_data['y']

dummy_cov = statistics.covariance(dummy_x_vals, dummy_y_vals)
dummy_x_mean = statistics.mean(dummy_x_vals)
dummy_y_mean = statistics.mean(dummy_y_vals)
print('dummy_cov: ', dummy_cov)
print("dummy x mean: ", dummy_x_mean)
print("dummy y mean: ", dummy_y_mean)



cov:  3214.7648484848482
x mean:  49.5
y mean:  227.703
dummy_cov:  4.143981818181821
dummy x mean:  16.269
dummy y mean:  14.982


In [6]:
scales = [0]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.cov = nn.Parameter(data = torch.tensor(dummy_cov), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(dummy_x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(dummy_y_mean), requires_grad = False)
    def forward(self,X,Y):
        # print("x size: ", X.size()[1])
        #  need to enforce same length, not yet
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<=torch.abs(0.01*X.size()[1]*(self.x_mean))
        y_mean_cons = torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<=torch.abs(0.01*Y.size()[1]*(self.y_mean))
        return (torch.logical_and(torch.logical_and(x_mean_cons,y_mean_cons), torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(X.size()[1]-1)*(self.cov))<0.01*(X.size()[1]-1)*(self.cov)), self.cov)



verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(real_cov)
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.cov = nn.Parameter(data = torch.tensor(real_cov), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(y_mean), requires_grad = False)
    def forward(self,X,Y):
        # print("x size: ", X.size()[1])
        #  need to enforce same length
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<=torch.abs(0.01*X.size()[1]*(self.x_mean))
        y_mean_cons = torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<=torch.abs(0.01*Y.size()[1]*(self.y_mean))
        return (torch.logical_and(torch.logical_and(x_mean_cons,y_mean_cons), torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(X.size()[1]-1)*(self.cov))<0.01*(X.size()[1]-1)*(self.cov)), self.cov)

# note scale = 2, or 3 makes it more precise, but too big.

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory output:  tensor(3214.7649)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-448,6246],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":13120,"total_assignments":827,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0,0],"module_sizes":{"kzg":[],"poseidon":[13120,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 3.663957118988037 seconds
Theory output:  tensor(3214.7649)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 3215.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[14955570959218682635, 4667139652385906200, 12836539004462631467, 1774684518626433649], [4224417983558473805, 851357164555783563, 5363851773531956453, 1448631618362554917], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5743885005642251665, 3430503978676436355, 7149667244725939006, 2902673458086333540]]], 'proof': '09f31a308bb02ed2663ca4b9af3f0dd1bee0126b250e0a9341f158901177c3a61383e691b48fd8a0b1180ae7d907a02f8467bafcba5675e77274185f93a3c0520317ca3e628360b6c0a6ce277f259ad80c13674f2cb30ccc12647142589ace7323086df037f2f7fa2189bd962feec853d12d586c308d425bcbc69e7488a149a11f41c138c2dab61b54439029f1d7119c4ef1c3176d29f32dad8b413c5d950e18148f7d46cb315e67269ded0c7ae98272007bea8ceaf68a5f7f0b64790b078f4a03ab8468919687ead541bc0933800935772927d77489369c58a30f84df7717912fd8bd7ed5c8628b941e7592c5e0f30afa0bcb026ae5241a2d950b2b78da96601380ba104d1ef7e5156473213d4908f90d7d2bdbee93ef004cd526e6f8395f5a276069c4e89f4664bb8415bdc3bf1b900d13006609

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

3215.0